# Environement set up details

## Installing locally (rather than colab)

Using `uv` (from https://docs.astral.sh/uv/)

```
# install uv in its own global location (using pipx)
pipx install uv
# create a virtual environment
uv venv
# activate the environment
source .venv/bin/activate
# install the Jupyter notebook packages
uv pip install ipykernel jupyter notebook
# install required packages
uv pip install numpy pandas matplotlib


In [ ]:
# imports for the project

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib.colors import ListedColormap

## Computation Graph of a Perceptron

![Computation Graph of a Perceptron](images/perceptron/perceptron.png)

The character 𝚫 (Delta) represents a change or update in the weight w_k. This represents the adjustment made to the weights during the training process.

Where:
- x_k: The input feature corresponding to the weight w_k.
- w_k: The weight w_k of the perceptron.
- y is the known desired  output.
- ŷ (y_hat) is the predicted output. 
- 𝚫w_k: The change in the weight w_k.
- Ƞ (eta): The learning rate, which controls the size of the weight updates.
- error: The difference between the predicted value (ŷ) and the desired  value (y).

Components:
- Aggregate function: a summation of the outputs of x . w. 
- Threshold function: sets the predicted value (ŷ) to 1 or -1 depending on the output of the aggregation function.


In [ ]:
# code comes from Sebastian Raschka's Python Machine Learning book


class Perceptron(object):
    """Perceptron classifier.

    Parameters
    ------------
    eta : float
        Learning rate (between 0.0 and 1.0)
    n_iter : int
        Passes over the training dataset

    Attributes
    ------------
    w_ : 1d-array
        Weights after fitting
    errors_ : list
        Number of misclassifications in every epoch

    """

    def __init__(self, eta=0.01, n_iter=50):
        self.eta = eta
        self.n_iter = n_iter

    def fit(self, X, y):
        """
        Training function that implements the perceptron learning rule.

        1. Initialize weights to zero
    
        For each epoch:

            2. For each sample:
                - Make prediction
                - Calculate error
                - Update weights if prediction is wrong
            3. Track number of errors per epoch
        
        4. return the instance of the Perceptron object

        Parameters
        ------------
        X : {array-like}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        y : array-like, shape = [n_samples]
            Target values.

        Returns
        ------------
        self : object

        """

        self.w_ = np.zeros(1 + X.shape[1])
        self.errors_ = []

        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X, y):
                update = self.eta * (target - self.predict(xi))
                self.w_[1:] += update * xi
                self.w_[0] += update
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self

    def net_input(self, X):
        """
        The Aggregation Function - Calculates the weighted sum of inputs (dot product).
        Formula: z = w0 + w1x1 + w2x2 + ... + wnxn
        where w0 is the bias unit and w1...wn are the weights
        """
        # uses numpy's dot method to calculate the dot product of the inputs X and the weights w_
        return np.dot(X, self.w_[1:]) + self.w_[0]

    def predict(self, X):
        """
        The Threshold function - Implements the unit step function (threshold).
        Returns:
        1 if net input >= 0
        -1 if net input < 0
        """
        # uses numpy's where method to return 1 if net_input is greater than or equal to 0, otherwise -1
        return np.where(self.net_input(X) >= 0.0, 1, -1)


## Training a perceptron model on the Iris dataset

doanload the data

In [ ]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)

df.tail()

extract and plot the data

In [ ]:
# Extract the first hundred class labels (it is known that 50 are Iris-setosa and 50 are Iris-virginica)
y = df.iloc[0:100, 4].values
# update the class labels to -1 or 1
y = np.where(y == 'Iris-setosa', -1, 1)
# Extract the first hundred features in columns 0 and 1, representively representing sepal length and petal length 
X = df.iloc[0:100, [0, 2]].values

# now plot the data
plt.scatter(X[:50, 0], X[:50, 1], color='red', marker='o', label='setosa')
plt.scatter(X[50:100, 0], X[50:100, 1], color='blue', marker='x', label='versicolor')
plt.xlabel('sepal length [cm]')
plt.ylabel('petal length [cm]')
plt.legend(loc='upper left')
plt.show()

note: this is clearly a linearly seperable problem

now we train the perceptron

In [ ]:
ppn = Perceptron(eta=0.1, n_iter=10)
ppn.fit(X, y)

# plot the errors per epoch

plt.plot(range(1, len(ppn.errors_) + 1), ppn.errors_, marker='o')
plt.xlabel('Epochs')
plt.ylabel('Number of misclassifications')
plt.show()

The perceptron converged after 6 epochs

Let's plot the decision boundry now

## Plotting the Decision Boundary

### Original plotting of the decision boundary

I include this code for completeness, it is directly from the book and may mean something for you. However I did not understand it immediately. 

I have been on a short journey to understand that the function is creating a grid of feature values and then running each value through the Perceptron's predict method to return the label for that point on a grid.

This creates a data set of 3 dimensions (petal length, sepal length, and label). The function uses a contour plot to represent the 3rd dimenstion (the label) as a colour.

The methods used to do this (specifically `arange`, `meshgrid`, and `ravel`) are from numpy and are new to me. I took some time to practical understand what they are doing, I include that journey after the original code for my reference and to help others. 

In [ ]:
# function to plot the inputs (X), the labels (y), 
def plot_decision_regions(X, y, classifier, resolution=0.02):
    """
    Plot decision regions for a classifier in a 2D feature space.

    Parameters:
    X : array-like, shape = [n_samples, n_features]
        Feature matrix.
    y : array-like, shape = [n_samples]
        Target vector.
    classifier : object
        Trained classifier with a predict method.
    resolution : float, optional (default=0.02)
        Resolution of the mesh grid used to plot the decision surface.

    Returns:
    None

    This function visualizes the decision boundaries of a classifier by plotting
    the decision surface and the data points in a 2D feature space. Different
    markers and colors are used for different classes.
    """
    # setup marker generator and color map
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1

    x1_range = np.arange(x1_min, x1_max, resolution)
    x2_range = np.arange(x2_min, x2_max, resolution)

    # Generate the meshgrid using np.meshgrid
    xx1, xx2 = np.meshgrid(x1_range, x2_range)

    # 
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)

    # use a filled contour plot to represent the 3d data (features petal (y) and sepal (x) lengths plus predicted class (z))
    # in a 2d plot. in this plot the z axis becomes the colours. 
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)

    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    # plot class samples
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1],
                    alpha=0.8, c=[cmap(idx)],
                    marker=markers[idx], label=cl)

In [ ]:
plot_decision_regions(X, y, classifier=ppn)

plt.xlabel('sepal length (cm)')
plt.ylabel('petal length (cm)')
plt.legend(loc='upper left')
plt.show()


### Understanding the numpy methods that plot the decision boundary

Below we can see that arange is creating an array from the first parameter to the second parameter, each value is the value before incremented by the third parameter until it arrives at the end.  

In [ ]:

# Generate a range of values from 0 to 10 with a step size of 1
range_1 = np.arange(0, 10, 1)
print("Range with step size 1:", range_1)

# Generate a range of values from 0 to 10 with a step size of 0.5
range_2 = np.arange(0, 10, 0.5)
print("Range with step size 0.5:", range_2)

# Generate a range of values from 0 to 10 with a step size of 2
range_3 = np.arange(0, 10, 2)
print("Range with step size 2:", range_3)

In [ ]:

# Define the min and max values for the ranges
feature_1_min, feature_1_max = 1, 5
feature_2_min, feature_2_max = 10, 20
resolution = 1

# Generate the ranges using np.arange
feature_1_range = np.arange(feature_1_min, feature_1_max, resolution)
feature_2_range = np.arange(feature_2_min, feature_2_max, resolution)

print("feature_1_range:", feature_1_range)
print("feature_2_range:", feature_2_range)

Now we have the ranges for each feature we can conbine them to make a grid. 

Remember the point of this is to create a contour (like a map) of the label value for each point on a feature grid.

Numpy has a feature called `meshgird` that is used to create two arrays of the feature values for each point on the grid.

In [ ]:
# Generate the meshgrid using np.meshgrid
feature_1_values, feature_2_values = np.meshgrid(feature_1_range, feature_2_range)

print("feature_1_values:\n", feature_1_values)
print("feature_2_values:\n", feature_2_values)

We can now visualise the grid. 

In [ ]:
# Visualize the meshgrid
plt.figure(figsize=(8, 6))
plt.scatter(feature_1_values, feature_2_values, s=50, color='blue')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Meshgrid Visualization')
plt.show()

Next we need to flatten the values and then combine them to into an array that represents the feature grid points 

In [ ]:
# Apply ravel to flatten the meshgrid arrays
feature_1_ravel = feature_1_values.ravel()
feature_2_ravel = feature_2_values.ravel()

# Print the flattened arrays
print("xx1_ravel:", feature_1_ravel)
print("xx2_ravel:", feature_2_ravel)

# Combine the flattened arrays and transpose
feature_grid_points = np.array([feature_1_ravel, feature_2_ravel]).T

# Print the combined and transposed array
print("grid_points:\n", feature_grid_points)

### Descriptive function to plot the decision boundary

Now that the different numpy methods have been tested on more vanilla data it should be clear. If it isn't change the original values for these

```
# Define the min and max values for the ranges
feature_1_min, feature_1_max = 1, 5
feature_2_min, feature_2_max = 10, 20
resolution = 1
```

By changing these it is possible to get a clearer view of the feature gird that the numpy methods are creating. 

Next the use of the classifier is renamed. This was important to me as, initially I'd thought that the function was doing the classification for the inputs (i.e. X). However X is only being used to get the minimum and maximum for the features. 

If this isn't clear, it may be worth changing `plot_decision_regions` to accept the min and max of the features rather than the whole data range.

In going over this bit of code:

```python

    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
```

the varaibles have been changed to these names, highlighting that the classifier is being used to values for the feature grid (therefore not for X) 

```python
    
    # use the classifier to calculate the label at each point on the gird
    label_per_point_on_feature_grid = classifier.predict(feature_grid)
    # Reshape the predictions back to match our grid dimensions for plotting
    label_per_point_on_feature_grid = label_per_point_on_feature_grid.reshape(feature_1_values.shape)
```

In [ ]:

def plot_decision_regions(X, y, classifier, resolution=0.02):
    """
    Plot decision regions for a classifier in a 2D feature space.

    Parameters:
    X : array-like, shape = [n_samples, n_features]
        Feature matrix.
    y : array-like, shape = [n_samples]
        Target vector.
    classifier : object
        Trained classifier with a predict method.
    resolution : float, optional (default=0.02)
        Resolution of the mesh grid used to plot the decision surface.

    Returns:
    None

    This function visualizes the decision boundaries of a classifier by plotting
    the decision surface in a 2D feature space.
    """
    # setup marker generator and color map
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])
    
    # calculate a grid of the features (feature_1 is the sepal length and feature_2 is the petal length)
    
    # firstly get the min and max of each feature
    feature_1_min, feature_1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    feature_2_min, feature_2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    # Create a fine grid of points covering our feature space so we can visualize
    # how the perceptron classifies every possible combination of feature values
    feature_1_values, feature_2_values = np.meshgrid(np.arange(feature_1_min, feature_1_max, resolution),
                                                     np.arange(feature_2_min, feature_2_max, resolution))
    # now flatten the 2 dimensional arrays into a one dimension array for both features
    # the array is transposed so that each point in the grid is represented as a row
    feature_grid = np.array([feature_1_values.ravel(), feature_2_values.ravel()]).T
    
    # use the classifier to calculate the label at each point on the gird
    label_per_point_on_feature_grid = classifier.predict(feature_grid)
    # Reshape the predictions back to match our grid dimensions for plotting
    label_per_point_on_feature_grid = label_per_point_on_feature_grid.reshape(feature_1_values.shape)

    # plot the decision surface

    # Create a filled contour plot where different colors show the different predicted classes
    # alpha=0.4 makes the coloring semi-transparent so we can see the data points
    plt.contourf(feature_1_values, feature_2_values, label_per_point_on_feature_grid, alpha=0.4, cmap=cmap)
    # Set the plot limits to show the full decision boundary region
    plt.xlim(feature_1_values.min(), feature_1_values.max())
    plt.ylim(feature_2_values.min(), feature_2_values.max())



To further higlight that this function is just for plotting the decision boundary the plotting of the features (X) and tehir corresponding labels (y) has been moved to a seperate function called `plot_class_samples`.

In [ ]:
def plot_class_samples(X, y, label_names):
    """
    Plot class samples in a 2D feature space.

    Parameters:
    X : array-like, shape = [n_samples, n_features]
        Feature matrix.
    y : array-like, shape = [n_samples]
        Target vector.
    label_names : list
        List of label names corresponding to the unique classes in y.

    Returns:
    None

    This function plots the class samples in a 2D feature space with a legend.
    """
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])
    
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1],
                    alpha=0.8, c=[cmap(idx)],
                    marker=markers[idx], label=label_names[idx])
    plt.legend()

Brining it all together we are able to plot the decision boundary and the samples, I have added the axis labels as well.

In [ ]:
plot_decision_regions(X, y, classifier=ppn)
label_names = ['setosa', 'versicolor']
plot_class_samples(X, y, label_names)
# Label the axes with the feature names to show what we're plotting
plt.xlabel('Sepal length [cm]')
plt.ylabel('Petal length [cm]')
plt.show()